## Application of ECOSTRESS Data: Exploring the imapcts of Urban Heat Islands

ADD NOTES HERE

### Step 1. Setup notebook

First, we need to import packages and set up some environment variables for the notebook.

In [ ]:
# Import Packages
import os, time, shutil
import warnings

import cartopy.crs as ccrs
import rioxarray as rxr
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
import geoviews as gv
import geopandas as gpd

# Some cells may generate warnings that we can ignore. Comment below lines to see.
warnings.filterwarnings('ignore')

# Set some default plotting options.
size_opts = dict(frame_width=800, frame_height=600, fontscale=1.5)
map_opts = dict(
    geo=True, tiles='EsriImagery',
    rot=90,
    xlabel='Longitude', ylabel='Latitude')

#### Transfer project data from the cloud

We want to copy over data stored in CyVerse so that we can access files quickly for analysis. To do this, we will use Python "shutil" package to copy files from the CyVerse data store to a local, temporary location in the HYR-SENSE GitHub repository. The data we are copying is stored in a shared ESIIL / HYR-SENSE location. We can copy the files just from the "Agriculture" module which contains data for this exercise. However, if you want to work with other modules, they can be accessed here as well.

In [ ]:
### Setup notebook data and copy to scratch/working folder
# Identify the location of the HYR-SENSE "data store"
data_store_path = '/data-store/iplant/home/shared/esiil/HYR_SENSE/data/01-Urban-Heat-Island'
# Set a destination path (this is a 'local' and temporary path)
dest = os.path.join(os.path.expanduser("~"),"HYR-SENSE","data","Urban_heat")
if not os.path.exists(dest):
    os.mkdir(dest) # create the directory for the copied data, if needed
    
# Using 'shutil' package, copy all the files over
shutil.copytree(data_store_path, dest, dirs_exist_ok=True)

print(" ")
print(" ")
print("*** Copy complete! ***")
print(" ")

### Step 2. Load Data

ADD NOTES HERE

In [ ]:
### Import the City of Boulder boundary GIS layer

# Define a path to the boundary
boulder_path = os.path.join(os.path.expanduser("~"),"HYR-SENSE",
    # GIS directory
    'data', 'Urban_heat',
    # City of Boulder file
    'City_of_Boulder_City_Limits.zip'
)

# Read file and merge geometries
boulder_gdf = gpd.read_file(boulder_path).dissolve()

# Check that we have a GeoDataFrame
boulder_gdf

In [ ]:
### Load the tree inventory data
# Define a path to the open tree data
tree_path = os.path.join(os.path.expanduser("~"),"HYR-SENSE",
    # GIS directory
    'data', 'Urban_heat',
    # City of Boulder file
    'Tree_Inventory_Open_Data.zip'
)
# Read file and merge geometries
tree_gdf = gpd.read_file(tree_path).dissolve()

# Check that we have a GeoDataFrame
tree_gdf

In [ ]:
### Load the example ECOSTRESS layer covering Boulder, CO

# Define the filename to use:  ECOSTRESS L2T LSTE
eco_file = 'ECOv002_L2T_LSTE_28527_009_13TDE_20230718T081442_0710_01_LST.tif'
# Define ECOSTRESS file path
data_dir = os.path.join(os.path.expanduser("~"),"HYR-SENSE","data","Urban_heat")

# Open the LSTE file using open_rasterio from the rioxarray library
eco_path = os.path.join(data_dir,eco_file)
eco_lst_ds = (
    rxr.open_rasterio(eco_path)
    # There is only 1 band, so we can squeeze and remove the band dimension.
    .squeeze('band', drop=True)
)
eco_lst_ds

### Step 3. Prepare the ECOSTRESS data for analysis

ADD NOTES HERE

Let's start by looking at the GIS layer we loaded that delineates the city limits of Boulder, CO

In [ ]:
### Plot the GIS boundary for Boulder, Colorado
(
    boulder_gdf.hvplot(
        **map_opts,
        line_color='orange', line_width=3,
        fill_color='white', fill_alpha=.05,
    )
    *
    gpd.GeoDataFrame(geometry=boulder_gdf.envelope).hvplot(
        **map_opts,
        line_color='skyblue', fill_color=None
    )
).opts(**size_opts)

In [ ]:
### Plot the tree locations
trees_plot = tree_gdf.hvplot.points(
    **map_opts,
    size=15
).opts(**size_opts)
trees_plot

Now let's load the ECOSTRESS image to show what data we will be working with in this notebook

In [ ]:
### Load the ECOSTRESS data
(
    eco_lst_ds
    .rio.reproject('EPSG:4326')
    .hvplot.image(
        x='x', y='y', **size_opts, 
        cmap='inferno', tiles='ESRI', 
        xlabel='Longitude', ylabel='Latitude', 
        title='ECOSTRESS LST (K)', 
        crs='EPSG:4326')
)


Above you can see a strong temperature gradients fro west to east moving from higher altitude to lower altitudes around the Boulder, CO area.  You will also see the data is presented in degrees Kelvin.  We can easily change degrees Kelvin to Degrees Celsius, which we will work with in the next steps 

In [ ]:
### Convert degrees Kelvin to degrees Celsius
eco_lst_ds_degC = eco_lst_ds
eco_lst_ds_degC.values = eco_lst_ds_degC.values-273.15
eco_lst_ds_degC

In [ ]:
### Confirm we have data in degrees C
(
    eco_lst_ds_degC
    .rio.reproject('EPSG:4326')
    .hvplot.image(
        x='x', y='y', **size_opts, 
        cmap='inferno', tiles='ESRI', 
        xlabel='Longitude', ylabel='Latitude', 
        title='ECOSTRESS LST (degC)', 
        crs='EPSG:4326')
)

Let's also overlay the GIS boundary on the ECOSTRESS data to ensure we have data that includes Boulder, CO

In [ ]:
### Load the ECOSTRESS data and overlay the GIS boundary
eco_plot = (
    eco_lst_ds_degC
    .rio.reproject('EPSG:4326')
    .hvplot.image(
        x='x', y='y', **size_opts, 
        cmap='inferno', tiles='ESRI', 
        xlabel='Longitude', ylabel='Latitude', 
        title='ECOSTRESS LST (degC)', 
        crs='EPSG:4326')
)
gis_boundary_plot = (
    boulder_gdf.hvplot(
        **map_opts,
        line_color='black', line_width=5,
        fill_color='white', fill_alpha=.05,
    )
    *
    gpd.GeoDataFrame(geometry=boulder_gdf.envelope).hvplot(
        **map_opts,
        line_color='skyblue', fill_color=None
    )
).opts(**size_opts)
combined_plot = eco_plot * gis_boundary_plot
combined_plot

We can also plot the data with the Boulder Open Tree data

In [ ]:
### Plot again with the tree locations
eco_plot = (
    eco_lst_ds_degC
    .rio.reproject('EPSG:4326')
    .hvplot.image(
        x='x', y='y', **size_opts, 
        cmap='inferno', tiles='ESRI', 
        xlabel='Longitude', ylabel='Latitude', 
        title='ECOSTRESS LST (degC)', 
        crs='EPSG:4326')
)
gis_boundary_plot = (
    boulder_gdf.hvplot(
        **map_opts,
        line_color='black', line_width=5,
        fill_color='white', fill_alpha=.05,
    )
    *
    gpd.GeoDataFrame(geometry=boulder_gdf.envelope).hvplot(
        **map_opts,
        line_color='skyblue', fill_color=None
    )
).opts(**size_opts)
trees_plot = tree_gdf.hvplot.points(
    **map_opts,
    size=15
).opts(**size_opts)
combined_plot = eco_plot * gis_boundary_plot * trees_plot
combined_plot

### Step 4. Analyze the ECOSTRESS data

In [ ]:
#https://py.geocompx.org/05-raster-vector